<a href="https://colab.research.google.com/github/chenqianhe/LSTM/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install torchtext
!python -m spacy download en

import torch
from torch import nn, optim
from torchtext import data, datasets

print('GPU: ', torch.cuda.is_available())

torch.manual_seed(123)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
GPU:  True


In [2]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 180k/84.1M [00:00<00:51, 1.63MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 67.3MB/s]


In [3]:
print('len of train data: ', len(train_data))
print('len of test data: ', len(test_data))

len of train data:  25000
len of test data:  25000


In [4]:
print(train_data.examples[15].text)
print(train_data.examples[15].label)

['An', 'unusually', 'straight', '-', 'faced', 'actioner', 'played', 'by', 'a', 'cast', 'and', 'filmed', 'by', 'a', 'director', 'who', 'obviously', 'took', 'the', 'material', 'seriously', '.', 'Imperfect', ',', 'as', 'is', 'to', 'be', 'expected', 'from', 'a', 'film', 'clearly', 'shot', 'on', 'a', 'tight', 'budget', ',', 'but', 'the', 'drama', 'is', 'involving--', 'it', "'s", 'one', 'of', 'those', 'films', 'that', 'when', 'it', 'gets', 'repeated', 'ad', 'nauseum', 'on', 'Cinemax', '2', 'or', 'More', 'Max', 'or', 'whatever', 'they', 'call', 'it', ',', 'you', 'end', 'up', 'watching', '40', 'minute', 'blocks', 'when', 'you', "'re", 'supposed', 'to', 'be', 'going', 'to', 'work', '.', 'Along', 'W/', '"', 'Deathstalker', '2', '"', ',', '"', 'Chopping', 'Mall', '"', ',', 'and', '"', 'The', 'Assault', '"', ',', 'a', 'reminder', 'that', 'Wynorski', 'is', 'a', 'much', 'more', 'talented', 'director', 'than', 'many', 'of', 'his', 'fellow', 'low', '-', 'budget', 'brethern', ',', 'who', 'has', 'a', 'r

In [5]:
TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.100d')
LABEL.build_vocab(train_data)



.vector_cache/glove.6B.zip: 862MB [06:25, 2.23MB/s]                          
100%|█████████▉| 399186/400000 [00:14<00:00, 27060.30it/s]

In [0]:
batchsz = 80
device = torch.device('cuda')
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size = batchsz,
    device = device
)

In [0]:
class Net(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim):
    super(Net, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=2,
                       bidirectional=True, dropout=0.5)
    self.fc = nn.Linear(hidden_dim*2, 1)
    self.dropout = nn.Dropout(0.5)
  
  def forward(self, x):
    embedding = self.dropout(self.embedding(x))

    output, (hidden, cell) = self.rnn(embedding)

    hidden = torch.cat([hidden[-2], hidden[-1]], dim=1)

    hidden = self.dropout(hidden)
    out = self.fc(hidden)
    return out

In [8]:
rnn = Net(len(TEXT.vocab), 100, 256)

pretrained_embedding = TEXT.vocab.vectors
print("pretrained_embedding: ", pretrained_embedding.shape)
rnn.embedding.weight.data.copy_(pretrained_embedding)
print('embedding layer inited.')

pretrained_embedding:  torch.Size([10002, 100])
embedding layer inited.


In [13]:
optimizer = optim.Adam(rnn.parameters(), lr=1e-3)
criteon = nn.BCEWithLogitsLoss().to(device)
rnn.to(device)

Net(
  (embedding): Embedding(10002, 100)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [0]:
def train(rnn, iterator, optimizer, criteon):
  avg_acc = []
  rnn.train()

  for i, batch in enumerate(iterator):
    pred = rnn(batch.text).squeeze(1)
    loss = criteon(pred, batch.label)
    acc = binary_acc(pred, batch.label).item()
    avg_acc.append(acc)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

In [0]:
import numpy as np

def binary_acc(preds, y):
  preds = torch.round(torch.sigmoid(preds))
  correct = torch.eq(preds, y).float()
  acc = correct.sum()/len(correct)
  return acc

def eval(rnn, iterator, criteon):
  avg_acc = []
  rnn.eval()
  with torch.no_grad():
    for batch in iterator:
      pred = rnn(batch.text).squeeze(1)
      loss = criteon(pred, batch.label)
      acc = binary_acc(pred, batch.label).item()
      avg_acc.append(acc)
  avg_acc = np.array(avg_acc).mean()
  print('>>test : ', avg_acc)

In [24]:

for epoch in range(10):
  eval(rnn, test_iterator, criteon)
  train(rnn, train_iterator, optimizer, criteon)

>>test :  0.4998402637652696
>>test :  0.6479632693548172
>>test :  0.6702875513047837
>>test :  0.866932919421516
>>test :  0.8763179030662147
>>test :  0.8869808428584577
>>test :  0.8875798842015739
>>test :  0.8851437821936684
>>test :  0.8819888317927765
>>test :  0.8856230155347635
